In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.utils import random
from sklearn.model_selection import train_test_split

In [4]:
class FFNetPytorch(nn.Module):
  def __init__(self, input_size, output_size, hl1=256, hl2=256):
    super(FFNetPytorch, self).__init__()
    '''
    Define the layers of the neural network. One hidden layer and output layer.
    The activation function used in between the two layers is sigmoid.
    '''
    self.layer1 = nn.Linear(input_size, hl1, bias = True)
    self.layer2 = nn.Linear(hl1, hl2, bias = True)
    self.layer3 = nn.Linear(hl2, output_size, bias=True)
    self.double()
    

  def forward(self, x):
    '''
    :param x: input to the model (N, NUM_FEATURES)

    :return:
      output: logits of the last layer of the model 
    '''
    x = torch.sigmoid(self.layer1(x))
    x = torch.sigmoid(self.layer2(x))
    x = self.layer3(x)

    return x

class CustomDataset(Dataset):
  def __init__(self, X, Y):
    self.X = X
    self.Y = Y

  def __getitem__(self, i):
    return self.X[i, ...], self.Y[i]
  
  def __len__(self):
    return self.X.shape[0]

In [5]:
#BECAUSE NO PXPY
import pandas as pd
INPUT_COLUMNS = ["start_speed", "spin_rate", "spin_dir", "zone", "pitch_type"]
CONTEXT_COLUMNS = ["b_score", "p_score", "b_count", "s_count", "outs", "pitch_num", \
    "on_1b", "on_2b", "on_3b", "inning", "p_throws", "stand", "top"]
OUTPUT_COLUMNS = ["at_bat_score"]

combined_data = pd.read_hdf("../source_files/model_data.hdf5")
inputs = combined_data.iloc[:, 0:-1].copy()
outputs = combined_data.iloc[:, -1].copy()

inputs_t = torch.tensor(combined_data.iloc[:, 0:-1].values)
outputs_t = torch.tensor(combined_data.iloc[:, -1].values)

model = torch.load("models/HL2_100_25.pt")

pitch = inputs.iloc[0, :].copy()
#print(pitch)
pitch["s_count"] = 2
pitch["pitch_num"] = 3

for zone in range(1,15):
    if zone==10: continue
    pitch["zone"] = zone
    print(f"Zone: {zone}, Result: {model(torch.tensor(pitch)).item()}")

Zone: 1, Result: 0.8693545782895769
Zone: 2, Result: 0.8711780125228579
Zone: 3, Result: 0.8737979771682081
Zone: 4, Result: 0.877535731754538
Zone: 5, Result: 0.8827492158357593
Zone: 6, Result: 0.8898040944653518
Zone: 7, Result: 0.899021441059239
Zone: 8, Result: 0.9105985695935699
Zone: 9, Result: 0.9245080611309318
Zone: 11, Result: 0.9575098516707863
Zone: 12, Result: 0.9747231620668866
Zone: 13, Result: 0.9906512453496819
Zone: 14, Result: 1.0038746564937158
